# **Exercise 3**: Build your own model for Charleston

In this exercise we will create our own SFINCS model for the area around Charleston. After doing this exercise you will know how to set up a basic SFINCS model from scratch, how to add forcing to it, run it and visualize the output. 

We will follow the following steps to create the model:

- Introduction: Add data to the HydroMT datacatalog
- Step 1: Setup the model domain
- Step 2: Initialize and define the model grid
- Step 3: Interpolate elevation data to the model grid
- Step 4: Set model active and boundary cells
- Step 5: Setup subgrid tables
- Step 6: Add forcing to the model
- Step 7: (Optional) Add observation (output) points
- Step 8: Save the model (and create zip archive)
- Step 9: Running the model
- Step 10: Visualize the output data

## Data used
 - Model domain:
    - self-defined polygons
 - Topography & Bathymetry:
    - GEBCO bathymetry
    - Local DEM for Charleston (10m resolution)
 - Waterlevels:
    - GTSM
 - Manning roughness:
    - ESA worldcover (land cover)
    - A reclassification table (reclassify land covert to manning roughness)

## Import dependencies

Before anything else we need to import the required packages

In [ ]:
import os
from pathlib import Path
import sys

import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import geopandas as gpd
import hydromt
import numpy as np
import matplotlib.pyplot as plt
from hydromt.log import setuplog
from hydromt_sfincs import SfincsModel

# local script imports
from sfincs_utils import run_sfincs, create_sfincs_model_archive


## Introduction: The hydroMT data catalog

HydroMT uses a data catalog to access and uniformize your data. A data catalog is a *.yml* file with references to external datasets, their metadata, together with some optional basic preprocessing, so the data is already in the correct format when loading it into python. Examples of preprocessing steps are are: renaming of variables, unit conversion using offsets and multiplication, setting missing nodata and CRS metadata.

For this course we have prepared the `"../data/data_catalog.yml"` file which references most data in the data folder, except for the local DEM file "topography_charleston_10m.tif". You can add this data by manually modifying the data catalog by editing the .yml file in a text editor such as VSCode or Notepad++ which have visual support for yml files. 

See the HydroMT documentation for more information on how to [prepare a data catalog](https://deltares.github.io/hydromt/v0.10.0/user_guide/data_prepare_cat.html) and [how to add a raster dataset](https://deltares.github.io/hydromt/v0.10.0/user_guide/data_types.html#rasterdataset). 
**NOTE**: make sure to use the version of the documentation matching your HydroMT installed version (likely v0.10) 

In short the steps to update the data catalog are:
- Open the ../data/data_catalog.yml file and look at the different entries. Note the different data types and drivers (i.e. methods to read the data)
- Now create a new data catalog key called `topography_charleston`, you can copy one of the other keys that are already present, e.g. the `gebco` one
- Navigate to the `../data` folder and change the path of your data to the correct file
- Also change the CRS to the correct coordinate reference system or leave it empty if it is contained in the file metadata

In [ ]:
# update the data catalog with to include the local DEM file and uncomment the following line
# data_catalog_file = "../data/data_catalog.yml"
data_catalog = hydromt.data_catalog.DataCatalog(data_catalog_file)

# view the data catalog, make sure the DEM is included
data_catalog.to_dataframe()[
    ["path", "data_type", "driver", "source_url", "source_version"]
]

## **Step 1:** Setup model domain

The first step of building a model is to specify the area we're interested in and turn this into the model domain.
The area of interest is best specified through a geojson file. An example is given at `../model_domain/model_domain.geojson`. 
A useful tool to draw your own geojson file is http://geojson.io or QGIS. 

Note that when including rainfall as flood driver you need to make sure that the watersheds contributing to pluvial flooding 
are completely within your model domain to make sure you capture all rainfall. In that case it is best to use shapefiles
of hydrological basins to guide your model domain delineation. In the data catalog the HydroSheds basin atlas is included
for the region of Charleston and can be used for this purpose. Here, we are focused on coastal flooding only and hence 
can be more relaxed about the inland boundary location. 

In [ ]:
# Add a (relative) path to the model domain geojson file.
# This file should contain the region of interest for the model.
# An example file is provided in the "../model_domain" folder.
# region_fn = Path(..)

# Load in our initial region of interest using geopandas
region = gpd.read_file(region_fn).to_crs(epsg=4326)

# Plot the region of interest
fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={"projection": ccrs.PlateCarree()})
ax.add_image(cimgt.OSM(), 10)
region.boundary.plot(ax=ax, color="red", label="Model Domain")
ax.legend()
plt.show()

## **Step 2:** Initialize and define the model grid

Now that we determined where to make a model, we can initialize and build our model.

When initializing the model, we can specify the following:
 - `root`: Folder where the model is to be saved. Path is relative to current working directory
 - `mode`: Whether you want to read/append/write the model (to prevent overwriting a model by accident)
 - `data_libs`: One or multiple data catalogs.
 - `logger`: The name of the logger used to keep track of changes to the model

If a model already exists in the root location we override it by initializing our new model in 'w+' mode. Note that we also add our data catalog to the model initialization

In [ ]:
# Specify root_folder and logger_name
# The root_folder is where the model will be stored, use a subfolder of the "../models" folder
root_folder = Path(r"../models/sfincs_charleston")
logger_name = "SFINCS_log_charleston"
logger = setuplog(
    logger_name, log_level=20
)  # log_level=20 is INFO, more messages are shown with log_level=10 (DEBUG)

# initialize model
sf = SfincsModel(
    root=root_folder,
    mode="w+",  # overwrite mode
    data_libs=[data_catalog_file],
    logger=logger,
)

Next we create a model grid by providing our domain and the grid resolution in units given by the crs. By specifying crs="utm", the closest UTM-zone will be used for your model. Any geometries specified in a CRS different than the model CRS will be converted automatically.

Initially the grid will be a rectangle around the provided domain. Later on we will refine this by defining the active cells.

In [ ]:
# CHOOSE YOUR MODEL RESOLUTION in the units of the projection (crs)
# We suggest to use a relatively coarse resolution for sake of the tutorial
# model_res = <SET MODEL RES> # in meters if model_crs is 'utm'

# we setup the grid based on the bounding box of the region
sf.setup_grid_from_region(
    region={"geom": region_fn},
    res=model_res,
    rotated=False,  # non-rotated grid.
    crs="utm",  # automatically the closest UTM zone is selected (unit is in meters),
)

# Inspect some grid paramaters:
# shape (mmax x nmax), and resolution in both directions (dx,dy)
print("\nGRID PARAMETERS\n")
print(f'shape: {sf.config["mmax"]}, {sf.config["nmax"]}')
print(f'resolution: {sf.config["dx"]}, {sf.config["dy"]}')
print(f'origin: {sf.config["x0"]}, {sf.config["y0"]}')
print(f'rotation: {sf.config["rotation"]}')
print(f'crs: EPSG:{sf.config["epsg"]}')

### Question 1

1a) What is the grid resolution?

1b) How many cells are there in x & y direction?

1c) Is the model rotated?

1d) Why are the grid origins (x0 & y0) not in latitude-longitude [degree] units?

#### Fill in your own answers:

1a)

1b)

1c)

1d)

## **Step 3:** Interpolate elevation data to the grid

We first interpolate elevation data to the grid using `setup_dep()`. This will help to refine the grid later.
Note that this data is not actually used in the model, as we will set up subgrid tables later on which replaces the grid-averaged elevation data.
However, having elevation data on the grid is useful to visualize the terrain and to define active cells in the next step.

It is possible to provide multiple datasets for the elevation data as we do below. Each subsequent data set will fill in the gaps in the grid. 
For each dataset, you can optionally set the valid area by providing a minimum or maximum elevation limit `zmin` and `zmax` or geometry `mask`. 
Furthermore, you can correct for differences in vertical datums between elevation datasets using a uniform or gridded `offset` value which is applied to the dataset.

#### Template for elevation data sources

The data is provided using the template for the `datasets_dep` input below. 
Per source, all arguments except for `elevtn` are optional

```python
datasets_dep = [
    {
        'elevtn': '<elevation data source name>',  # source 1
        'offset': '<offset to vertical datum>',
        'zmin': '<minimum valid elevation value>',
        'zmax': '<maximum valid elevation value>',
    },
    {
        'elevtn': '<elevation data source name>',  # source 2
    }
]
```

In [ ]:
# Specify the elevation datasets to use. Note that the names refer to the keys in the data_catalog
# The first dataset is used as the primary elevation source
# The second dataset is used to fill gaps in the first dataset
# charleston_dem_source = ADD YOUR DEM SOURCE HERE
datasets_dep = [
    {
        "elevtn": charleston_dem_source
    }, 
    {
        "elevtn": "gebco", 
        "zmax": 0
    }
]
dep = sf.setup_dep(datasets_dep=datasets_dep, buffer_cells=2)

In [ ]:
# note the elevation values are stored in the "dep" variable of the model grid component.
sf.grid

In [ ]:
# Plot the elevation on top of the model region and satellite image. The variable argument sets which model variable to plot
fig, ax = sf.plot_basemap(variable="dep", bmap="sat")
sf.grid.raster.vector_grid("lines").plot(ax=ax, color="k", linewidth=0.1, label="grid")
ax.legend()

## **Step 4:** Set active and boundary cells

### Active cells

Next, we refine the grid by specifying which cells are active by calling the function `setup_mask_active()`. 
Active cells are initialized based on a `mask` geometry. 

Below you are asked to define the active cells for your model in two steps to learn how the arguments work. 
In the first step you are asked to initialize the active cells based on the region geometry `mask` which you used the initialize the grid.
In the second step you can further refine it using additional arguments.

In [ ]:
# set mask based on mask 

# Set the mask based on the region mask
sf.setup_mask_active(
    mask=region_fn,  # Path to the mask file, commonly the region of interest
    reset_mask=True,   # Make sure we start with a clean mask
)

# Plot the mask. Using variable='msk' will display the mask values for the active cells.
_ = sf.plot_basemap(variable="msk", bmap="sat")

### Improving the mask

If you are not satisfied with the initial active mask you created, you can either modify the geometry file you used in e.g. QGIS or use the additional arguments of the `setup_mask_active()` function to refine it as shown below.

You can refine your mask using the a minimum `zmin` and maximum height `zmax` arguments.
Furthermore, he active cells can be refined by adding areas inside the `include_mask` geometries, or removing areas inside the `exclude_mask` geometries. 
Small groups of isolated cells can be removed with the `drop_area` argument while holes in the mask can be filled with the `fill_area` argument. 
If `reset_mask=True` to existing mask is re-initialized and previous edits are overwritten. 
Be mindful of this setting in case you define active cells with multiple calls of `setup_mask_active()`.

You can draw include and exclude geometries in e.g. http://geojson.io or QGIS and export to geojson to use here. 
If you want draw these geometries on top of the existing mask you can export it to geotiff using `sf.grid["msk"].raster.to_raster("path/to/mask.tif")` and load in QGIS.

Find more information about the function `setup_mask_active()` [in the docs](https://deltares.github.io/hydromt_sfincs/stable/_generated/hydromt_sfincs.SfincsModel.setup_mask_active.html)


In [ ]:
# Include mask and exclude mask polygons:
# zmin_active = <ZMIN LEVEL>
# zmax_active = <ZMAX LEVEL>
# include_mask_filename = Path('../model_domain/YOUR_INCLUDE_MASK')
# exclude_mask_filename = Path('../model_domain/YOUR_EXCLUDE_MASK')
# drop_area = <YOUR_DROP_AREA>
# fill_area = <YOUR_FILL_AREA>

# Run function:
# comment out a line if you don't use the include mask, exclude mask, drop area or fill area
sf.setup_mask_active(
    zmin=zmin_active,  # Minimum elevation [m] to set as active
    zmax=zmax_active,  # Maximum elevation [m] to set as active
    include_mask=include_mask_filename,
    exclude_mask=exclude_mask_filename,
    drop_area=drop_area,
    fill_area=fill_area,
    reset_mask=False,  # Make sure we continue with the current mask
)

# Plot the mask. Using variable='msk' will display the mask values for the active cells.
_ = sf.plot_basemap(variable="msk", bmap="sat")

### Question 2

2a) Inspect the mask that is created, does is comply with your expectations? 

2b) How many active cells does your model have? (Tip: take the sum of all mask cells larger than zero using `sf.grid['msk']`)

#### Fill in your own answer:

2a)

2b)

### Boundary cells

The boundaries function by default as hard walls, allowing no flow in or out. We want to use the boundaries on *downstream* open water boundaries to force the water levels there. We can do this by calling `setup_mask_bounds()`. Setting `btype="waterlevel"` will specify waterlevel boundaries (msk=2). Boundary cells can be selected similarly to the active cells, by providing elevation limits `zmax` (i.e. all boundary cells below this elevation will set as waterlevel boundary) and/or using `include_mask` and `exclude_mask`.

If you suspect any significant flow to outside the model to occur at some boundaries, you can set those boundaries as outflow boundaries by setting `btype="outflow"` providing the geometry of the desired bounds. This is usually only necessary when applying rainfall to the grit. For stability, avoid outflow boundaries adjacent to waterlevel boundaries.  

Find more information about the function `setup_mask_bounds()` [in the docs](https://deltares.github.io/hydromt_sfincs/stable/_generated/hydromt_sfincs.SfincsModel.setup_mask_bounds.html)

In [ ]:
# Setup water level bounds

# use either on of the following options:
# zmax_bounds = <THE_MAX_WATERLEVEL>
# waterlevel_mask_filename = Path('../models/YOUR_WATERLEVEL_MASK')

# comment out a line to ignore the argument
sf.setup_mask_bounds(
    btype="waterlevel",
    zmax=zmax_bounds,          # Maximum elevation of boundary cells [m] to assign as waterlevel boundary
    include_mask=waterlevel_mask_filename,  # mask of boundary cells to assign as waterlevel boundary
    reset_bounds=True,
)

# Inspect the updated mask. Mask value 2 means waterlevel boundary, mask value 3 means outflow boundary
_ = sf.plot_basemap(variable="msk", bmap="sat", plot_bounds=False)

### Question 3

a) Does the waterlevel boundary nicely cover coastal boundary of your model?


#### Fill in your answers:

3a)

## **Step 5:** Setup subgrid tables

By pre-calculating lookup tables that account for the subgrid variation in bedlevels and friction, the computation can be executed at a courser resolution (i.e. the model grid) without loosing much accuracy up to a certain resolution. 
The lookup tables contain storage and conveyance capacity levels for (usually ~15) different water levels. 
At computation the cell storage and conveyance capacity to neighboring cells is estimated by interpolating the information from these tables. 
The subgrid approach assumes a uniform water level within the entire model grid cell. Hence on steep terrain this assumption might not be valid. 

The subgrid tables require both the elevation `datasets_dep`, as specified in step 3, and roughness datasets `datasets_rgh` as shown below.
Furthermore, the subgrid resolution `nr_subgrid_pixels` needs to be specified as a ratio of the model grid resolution (i.e. 10 means 10x finer subgrid resolution).
The number of levels `nlevels` at which the subgrid variation is described can be specified. More levels usually lead to more accurate results, but also increase the model setup time and size of the model files.
For the computation of the subgrid tables, the maximum number of subgrid pixels per calculation block `nrmax` needs to be specified. Decrease this value if you run into memory errors during model setup.
For other options of the `setup_subgrid()` function see the [docs](https://deltares.github.io/hydromt_sfincs/stable/_generated/hydromt_sfincs.SfincsModel.setup_subgrid.html).

Reference paper: https://gmd.copernicus.org/articles/18/843/2025/ 



### Define model **surface roughness** data

We can specify the surface roughness to better characterize the water flow on open surfaces. When we don't specify anything defaults values for the manning roughness are used based on elevation thresholds. We can specify manning roughness values ourselves, or we can provide a dataset with landuse classes. These classes are then converted to roughness values using a reclassification table. Here, it is also possible to provide multiple datasets to fill in any gaps that might exist in the previous datasets, similar to the elevation. 

In this example we will use the ESA worldcover land use dataset together with a reclass table which maps each landuse class to a specific manning roughness value, see below how such table should look like. Both are called with the proper entry name from the data catalog. We recommend using more detailed local datasources if those are available. 

#### example reclass table to map landuse to roughness values (N)
| class | description               | N     |
|-------|---------------------------|-------|
| 10    | Tree cover                | 0.12  |
| 20    | Shrubland                 | 0.05  |
| 30    | Grasland                  | 0.034 |
| ..    | ........                  | ..... |
| 0     | No data                   | -999  |

#### Template for roughness data sources

The data is provided using the template for the `datasets_rgh` input below. 
Per source, the `lulc` and `reclass_table` fields are required.

```python
datasets_rgh = [
    {
        "lulc": <landuse/landcover source name>,  # source 1
        "reclass_table": <reclassification table source name>,  
    },
    {
        "lulc": <landuse/landcover source name>,  # source 2
        "reclass_table": <reclassification table source name>,  
    }
]
```

In [ ]:
# Specify the surface roughness datasets to use. Note that the names refer to the keys in the data_catalog
# Here we use the ESA worldcover landuse dataset together with a reclass table which maps each landuse class to a specific manning roughness value
datasets_rgh = [
    {
        "lulc": "esa_worldcover",  # landuse/landcover dataset
        "reclass_table": "esa_worldcover_mapping",  # reclassification table
    }
]

### Define model subgrid elevation

See step 3 for defining elevation data sources. You may use the same elevation datasets as in step 3 or different ones if you have higher resolution data available for your model domain.
In case of the latter, redefine the `datasets_dep` variable accordingly.

In [ ]:
# specify the subgrid resolution as a ratio of the model grid (i.e. 10 means 10x finer subgrid resolution)
# select a value that is appropriate based on your model grid resolution and the resolution of the available high resolution topography data
# To avoid running into memory errors, for this course we advise to use a subgrid resolution of at least 10 meters. 
# nr_subgrid_pixels = <SET SUBGRID RESOLUTION>

# NOTE: this step can take some time (~minutes)
sf.setup_subgrid(
    datasets_dep=datasets_dep,  # elevation datasets, same as step 3
    datasets_rgh=datasets_rgh,  # roughness datasets, same as step 4
    nlevels=15,  # number of levels at which the subgrid variotion is described
    nr_subgrid_pixels=nr_subgrid_pixels,  # subgrid resolution as ratio of model grid (i.e. 10 means 10x finer subgrid resolution)
    nrmax=4000,  # maximum number of subgrid pixels per calculation block (decrease if memory errors occur)
    write_dep_tif=True,  # write subgrid elevation to tif
)

### Question 4

a) What is the subgrid pixel resolution? 


## **Step 6:** Add Forcing

Next we configure the forcing, the data we use as input in the model. SFINCS forcings include water levels at the waterlevel boundaries, river discharges at source points, rainfall, wind and pressure. For this example where we simulate coastal flooding, we only use water level boundaries. First, we need to set a time window to simulate. In subsequent steps to setup the forcing data will only be saved for this time period. 

Look on the internet when a large flood event happened in Charleston in 2016 (we only have forcing data for October 2016 prepared) and use this event for your model by setting the start time `tstart` and end time `tend`, the reference time `tref` is usually also set to the start time. 2 days of simulation around the event should be sufficient. In addition we set the output time resolution at which gridded normal and max outputs are saved in *sfincs_map.nc* using `dtout` and `dtmaxout`, and the time resolution at which timeseries outputs are saved in *sfincs_his.nc* `dthisout`. Note that this is not the computational time step which is adaptive and calculated based on the courant number. Finally, you should set a uniform initial water level at all active cells `zsini`. Be cautious to not set this value too high to avoid initializing the model with flooding. You can set update more model settings in this step, for a full overview see [sfincs docs](https://sfincs.readthedocs.io/en/latest/parameters.html)


In [ ]:
# Specify the simulation time in the model config
# model_time_config = {
#     "tref": "YYYYMMDD hhmmss", #FILL IN THE REFERENCE TIME (can be any date)
#     "tstart": "YYYYMMDD hhmmss", #FILL IN THE START TIME OF THE SIMULATION
#     "tstop": "YYYYMMDD hhmmss", #FILL IN THE END TIME OF THE SIMULATION
#     "dtout": ..., # [sec] FILL IN THE TIMESTEP OF THE MAP OUTPUT
#     "dtmaxout": ...., # [sec] FILL IN THE TIMESTEP OF THE MAXIMUM MAP OUTPUT
#     "dthisout" : ..., # [sec] FILL IN THE TIMESTEP OF THE SCALAR OUTPUT
#     "zsini": ..., # [m] FILL IN THE INITIAL WATERLEVEL
# }

sf.setup_config(**model_time_config)

### Water level forcing

For the forcing of the water levels at the boundary we use the Global Tide and Surge Model (GTSM). This includes hourly waterlevels at predetermined points. 
To get the waterlevels at our boundary we interpolate these points to our boundary when we read in the data. 

Check the `gtsm_locations` and `gtsm_timeseries` files. You can find the file locations in the data catalog.
Note that the index attribute of the locations matches with the column names of the timeseries file. This is important for HydroMT to know which timeseries belongs to which location!

In [ ]:
# change to locations and timeseriesa
sf.setup_waterlevel_forcing(
    locations="gtsm_locations",
    timeseries="gtsm_timeseries",
    merge=False,  # do not merge with existing locations and timeseries
    # use a large buffer to include all points if the following error is raised
    # ValueError: The gdf_locs index and df_ts columns must be the same
    buffer=1e5,  # buffer from model boundary [m]; exclude points outside the buffer
)

In [ ]:
# The keys to access the model forcing
print(sf.forcing.keys())

# Plot the forcing
_ = sf.plot_forcing()

In [ ]:
# Plot model summary so far, showing the GTSM points used for the waterlevel forcing
_ = sf.plot_basemap(variable="dep", bmap="sat")

### Question 5

a) For how long are you running the simulation?

b) How many waterlevel points are used to force the model?

c) Which cells are used to force the model with waterlevels? And how are the waterlevels calculated for these cells? (hint: look at the [online documentation](https://sfincs.readthedocs.io/en/stable/input_forcing.html#water-levels))

### Fill in your answers:

a)

b)

c)

## **Step 7:** Observation points and lines (BONUS)

The final output of the model will generally not be stored with the same temporal resolution the model uses during calculations to keep the output size in check. We can however specify some points and lines, called observation points and lines for which the model outputs are tracked with the higher temporal resolution. Observation points track the water level, while observation lines track the flow volumes passing through that line. These timeseries outputs will be saved in *sfincs_his.nc*

The geometry of the observation points and lines are  provided as geojson files. In our exercise we will only add observation points.
Create your own geojson by going to http://geojson.io and create observation points in SFINCS. 
To do so we use the `setup_observation_points` function, calling the geojson that you made.

For more information about this function the [docs](https://deltares.github.io/hydromt_sfincs/stable/_generated/hydromt_sfincs.SfincsModel.setup_observation_points.html).

In [ ]:
# Path to observation points & lines geometries
# obs_points_fn = Path(r'../model_domain/YOUR_OBS_POINTS_GEOJSON')

# Setup observation. merge argument handles whether to override or not
sf.setup_observation_points(locations=obs_points_fn, merge=False)

# Print the keys to access the observation geometries
sf.geoms.keys()

Finally we plot a summary of our model. This shows the boundary used as waterlevel, the points in the GTSM data used for the waterlevel forcing, and the observation points and lines.

In [ ]:
_ = sf.plot_basemap(variable="dep", bmap="sat")

## **Step 8:** Save model

Before running the model you need to save it. You can also update and finetune it later by reading it back in append mode. Fill in the cell below to save your model:

In [ ]:
# SAVE YOUR MODEL:
sf.write()

Saving the model will create a standard folder structure to be used when reading a model in. 
Note that the files in the GIS folder are not used by the model, but are useful for visualisation purposes.
This folder structure can be inspected with the code below or by looking at the folder structure in your file explorer:

In [ ]:
def print_directory_tree(directory):
    for root, _, files in os.walk(directory):
        level = root.replace(directory, "").count(os.sep)
        indent = " " * 2 * (level)
        print("{}{}/".format(indent, os.path.basename(root)))
        subindent = " " * 2 * (level + 1)
        for f in files:
            print(f"{subindent}+ {f}")


print_directory_tree(sf.root)

### create a zip archive of the model

Note this can be used to submit your model on canvas.

In [ ]:
# NOTE: the depfile and manningfile are not written to disk if a subgrid table is present, you may ignore this warning

create_sfincs_model_archive(
    sfincs_inp=Path(sf.root, "sfincs.inp"),
    zip_filename=Path(sf.root, "sfincs_model.zip"),
)

## **Step 9:** Run your model
We have now set up our own SFINCS model for Charleston, well done! We are now ready to run the model. 
For more infor see notebook `1_run_your_first_model.ipynb`

In [ ]:
# NOTE you need to download the SFINCS executable first (see first notebook) and update the path below
# sfincs_exe = Path("../../00_software", "SFINCS_v2.1.1", "sfincs.exe")

run_sfincs(
    Path(root_folder, "sfincs.inp"),  # path to the SFINCS model root folder
    run_method="exe",  # run SFINCS using the executable, other options are "docker" or "singularity"
    sfincs_exe=sfincs_exe,  # path to the sfincs executable if you want to run SFINCS on windows
)

## **Step 10:** Inspecting the results: floodmap
Inspect the results of the model run by plotting the maximum water depths on a map. 
In order to do so you should first read the results of the model (*sf.read_results()*). 
After this you can access the results by using the results attribute: *sf.results*.
The results you can plot using matplotlib.

*Tip: Print all available result keys by typing `print(sf.results.keys())`*  
*Tip:* You can also plot the results on top of the SFINCS basemap

In [ ]:
# hmax is computed by SFINCS and read-in from the sfincs_map.nc file
sf.read_results()

In [ ]:
# We use the subgrid resolution topography to downscale the maximum water level to water depth at the subgrid resolution
# First, we read the subgrid resolution topography and check if the file exists
dep_subgrid_file = Path(root_folder, "subgrid", "dep_subgrid.tif")  
dep_subgrid = hydromt.open_raster(dep_subgrid_file, nodata=np.nan) # elevation at subgrid resolution

# calculate the maximum water depth (hmax) as the difference between 
# the maximum water level (zsmax) and the elevation (dep)
zsmax = sf.results["zsmax"].max(dim="timemax") # maximum water level
zsmax_subgrid = zsmax.raster.reproject_like(dep_subgrid, method="nearest")  # reproject to subgrid resolution
hmax_subgrid = np.maximum(0, zsmax_subgrid - dep_subgrid).round(2).astype(np.float32) # maximum water depth
# Here, we assume that all areas with a negative depth are permanent water bodies
# This is a strong simplification. It is better to use a more sophisticated approach
# to identify permanent water bodies, e.g. based on a water mask or a "tide-only" simulation.
permament_water = dep_subgrid < 0 
hmax_subgrid = hmax_subgrid.where(~permament_water, np.nan) # mask areas with permanent water
# update (geospatial) metadata
hmax_subgrid.raster.set_crs(dep_subgrid.raster.crs)
hmax_subgrid.raster.set_nodata(np.nan)
hmax_subgrid.name = "max water depth"
hmax_subgrid.attrs["units"] = "m"

fig, ax = sf.plot_basemap(
    fn_out="hmax.png",  # save figure to mod.root/figs/hmax.png
    variable=hmax_subgrid,
    plot_bounds=False,
    bmap="sat",
    zoomlevel=12,
    figsize=(11, 7),
    vmin=0,  # set minimum value for colorbar
    vmax=3,  # set maximum value for colorbar
    cmap="BuPu",
)
ax.set_title("SFINCS maximum water depth")


In [ ]:
# save the results to a geotiff file
# round to 2 decimals to reduce file size

hmax_subgrid.raster.set_nodata(np.nan)
hmax_subgrid.round(2).raster.to_raster(
    Path(sf.root, "hmax.tif"),
    compress="lzw",
    overwrite=True,
    dtype="float32",
)

### Question 6

6a) Approximately how far inland does the coastal flood reach?

#### Fill in your answers:

6a)

## **What's Next:** Updating your model boundary conditions

Well done, you created your own SFINCS model! 